In [1]:
import os
import pandas as pd
import geopandas as gpd

In [2]:
main_dir = os.getcwd()
data_dir = os.path.join(main_dir, 'data')
res_dir = os.path.join(data_dir, 'Resources')
census_dir = os.path.join(res_dir, 'Census_Data')

In [8]:
st_tracts = gpd.read_file(os.path.join(data_dir, 'windshield_data.geojson'))

In [9]:
#st_tracts = st_tracts.to_crs(epsg=3452) # https://www.spatialreference.org/ref/epsg/3452/  (USES FEET)

In [10]:
st_tracts.columns

Index(['OBJECTID_1', 'Join_Count', 'TARGET_FID', 'OBJECTID', 'CENTERLINE',
       'FROMLEFT', 'TOLEFT', 'FROMRIGHT', 'TORIGHT', 'FULLNAME', 'FEDROUTE',
       'FEDRTETYPE', 'AFEDRTE', 'AFEDRTETYP', 'STROUTE', 'STRTETYPE', 'ASTRTE',
       'ASTRTETYPE', 'CTYROUTE', 'ONEWAYDIR', 'ROADLEVEL', 'INWATER',
       'ROADCLASS', 'MTFCC', 'OWNEDBY', 'MAINTBY', 'MUNILEFT', 'MUNIRIGHT',
       'STATELEFT', 'STATERIGHT', 'ZIPLEFT', 'ZIPRIGHT', 'MSAGLEFT',
       'MSAGRIGHT', 'ESNLEFT', 'ESNRIGHT', 'LASTUPDATE', 'LASTEDITOR',
       'FULLNAMEAB', 'JoinId', 'Shape_Leng', 'STATEFP10', 'COUNTYFP10',
       'TRACTCE10', 'GEOID10', 'NAME10', 'NAMELSAD10', 'MTFCC10', 'FUNCSTAT10',
       'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10', 'Shape_Le_1',
       'Shape_Length', 'geometry'],
      dtype='object')

In [11]:
st_tracts = st_tracts.drop(columns=['OBJECTID_1',
                                    'Join_Count',
                                    'TARGET_FID',
                                    'OBJECTID',
                                    'CENTERLINE',
                                    'FEDROUTE', 
                                    'FEDRTETYPE',
                                    'AFEDRTE',
                                    'AFEDRTETYP',
                                    'STROUTE',
                                    'STRTETYPE',
                                    'ASTRTE',
                                    'ASTRTETYPE',
                                    'CTYROUTE',
                                    'ROADLEVEL',
                                    'MTFCC',
                                    'OWNEDBY',
                                    'MAINTBY',
                                    'MUNILEFT',
                                    'MUNIRIGHT',
                                    'STATELEFT',
                                    'STATERIGHT',
                                    'ZIPLEFT',
                                    'ZIPRIGHT',
                                    'MSAGLEFT',
                                    'MSAGRIGHT',
                                    'ESNLEFT',
                                    'ESNRIGHT',
                                    'LASTUPDATE',
                                    'LASTEDITOR',
                                    'JoinId',
                                    'NAME10',
                                    'Shape_Leng',
                                    'STATEFP10',
                                    'COUNTYFP10',
                                    'TRACTCE10',
                                    'MTFCC10',
                                    'FUNCSTAT10',
                                    'ALAND10',
                                    'AWATER10',
                                    'Shape_Length',
                                    'Shape_Le_1',
                                    'INTPTLAT10',
                                    'INTPTLON10',
                                    'NAMELSAD10',
                                    'INWATER',
                                    'ONEWAYDIR'
                                    
                                   ]
                                    
                           )
st_tracts['CENSUS_TRACT'] = st_tracts['GEOID10']
st_tracts.drop(columns='GEOID10')

,FROMLEFT,TOLEFT,FROMRIGHT,TORIGHT,FULLNAME,ROADCLASS,FULLNAMEAB,Shape_Length,geometry,CENSUS_TRACT
0,7300,7398,7301,7399,West End Boulevard,Local,West End Blvd,107.486394,"LINESTRING (3667030.950 555545.256, 3667059.79...",22071013301
1,8600,8698,8601,8699,Pontchartrain Boulevard,Local,Pontchartrain Blvd,48.557767,"LINESTRING (3666584.278 555559.193, 3666583.43...",22071007606
2,7250,7298,7251,7299,West End Boulevard,Local,West End Blvd,44.872801,"LINESTRING (3667018.932 555418.901, 3667026.90...",22071013301
3,7200,7498,7201,7499,Lakeshore Drive,Major Arterial,Lakeshore Dr,448.101672,"LINESTRING (3666840.425 555358.892, 3666841.52...",22071013301
4,7200,7298,7201,7299,Regent Street,Local,Regent St,191.420409,"LINESTRING (3666255.079 555416.775, 3666253.87...",22071007606
...,...,...,...,...,...,...,...,...,...,...
27152,0,0,5500,5598,Morrison Road,Local,Morrison Rd,69.284448,"LINESTRING (3694069.843 557068.997, 3694261.59...",22071001720
27153,0,0,0,0,Morrison Road,Local,Morrison Rd,31.076633,"LINESTRING (3694069.843 557068.997, 3694049.01...",22071001720
27154,0,0,0,0,,,,0.000000,GEOMETRYCOLLECTION EMPTY,
27155,0,0,0,0,,,,0.000000,GEOMETRYCOLLECTION EMPTY,


In [13]:
st_tracts['Shape_Length'].sum()

3256048.1582908276

In [448]:
# Fix census tract numbers
geoid = []
for tract in st_tracts['CENSUS_TRACT']:
    geoid.append('1400000US' + str(tract))
st_tracts['GEO_ID'] = geoid
st_tracts.drop(columns=['CENSUS_TRACT'])

# Copy to be used to reset after merge is done
st_tracts_orig = st_tracts.copy()

In [551]:
def make_quartiles(data, column):
    data['quartile'] = pd.qcut(data[column], 4, labels=[0,1,2,3])

In [552]:
def get_sample(df):
    data = pd.DataFrame()
    chosen = []
    for i in range(4):
        qt = df.loc[df['quartile'] == i]
        samp = qt.sample()
        chosen.append(samp)
    data = pd.concat(chosen, ignore_index=True)
    return data

In [553]:
def get_coords(tracts):
    coords_lst = []
    for i in range(4):
        road = tracts[tracts['quartile'] == i]
        road = road.sample()
        coords = list(road.geometry.iloc[0].coords)
        coords_lst.append(coords[0])
    return coords_lst

In [554]:
def get_points():
    final_coords = []
    for i in variable_list:
        make_quartiles(i[0],i[1])
        sample = get_sample(i[0])
        st_tracts_merged = st_tracts.merge(sample, left_on='GEO_ID', right_on='id', how='left')
        variable_coords = get_coords(st_tracts_merged)

        # Flip the x,y values
        vc = variable_coords
        vc = [(vc[0][1], vc[0][0]), 
              (vc[1][1], vc[1][0]),
              (vc[2][1], vc[2][0]),
              (vc[3][1], vc[3][0])
             ]

        final_coords.append([i[1], vc])
    return final_coords

In [555]:
employ = pd.read_csv(os.path.join(census_dir, 'unemployed_percent2.csv'))
race = pd.read_csv(os.path.join(census_dir, 'race_clean2.csv'))
poverty = pd.read_csv(os.path.join(census_dir, 'poverty2.csv'))
income = pd.read_csv(os.path.join(census_dir, 'income_clean2.csv'))
vehicle = pd.read_csv(os.path.join(census_dir, 'vehicle_access_clean2.csv'))

variable_list = [[employ, 'UNEMPLOYMENT_OV16'], 
                [race, 'PERCENT_POP_BLACK_OR_AA_ALONE'],
                [poverty, 'PERCENT_BELOW_POVERTY'],
                [income, 'MEDIAN_EARNINGS_IN_DOLLARS_POPULATION_16_YEARS_AND_OVER_WITH_EARNINGS'],
                [vehicle, 'PERCENT_HOUSEHOLD_NO_VEHICLE']
                ]

In [593]:
output=[]
for i in range(20):
    points = get_points()
    output.append([i, points])

In [594]:
output

[[0,
  [['UNEMPLOYMENT_OV16',
    [(29.98545944866005, -90.12218982685286),
     (30.02457357666901, -90.05429492501544),
     (29.968516739438776, -90.1195311145475),
     (29.97472398786712, -90.01247979931888)]],
   ['PERCENT_POP_BLACK_OR_AA_ALONE',
    [(30.007395667258333, -90.11847827466146),
     (29.963602740526458, -90.07226637894273),
     (29.956911744993683, -90.13344907167216),
     (30.05390353201853, -89.96381057112357)]],
   ['PERCENT_BELOW_POVERTY',
    [(29.921884136297688, -90.12373209316019),
     (29.946540495698393, -90.05105615615808),
     (29.94056523575191, -90.09621295003288),
     (30.019664301322095, -90.00103460593071)]],
   ['MEDIAN_EARNINGS_IN_DOLLARS_POPULATION_16_YEARS_AND_OVER_WITH_EARNINGS',
    [(30.007119387379422, -90.02059782513412),
     (30.001015169923573, -90.04782339882658),
     (29.898991776210494, -90.0009718523199),
     (29.955007850505215, -90.04667402598012)]],
   ['PERCENT_HOUSEHOLD_NO_VEHICLE',
    [(30.051128017946198, -89.93069941

# write to file
with open('testing.txt', 'w') as file:
    for lst_item in final_coords:
        file.write(f'{lst_item}\n')

In [ ]:
final_coords